In [ ]:
import requests
import urllib
import json
import csv
import pandas as pd
import configparser
import json
from pandas.io.json import json_normalize
import os
import glob

In [ ]:
folder = 'PATH_TO_REPO/Review4Repair_mapped_code_with_comments/'
project_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]